In [3]:
# Package Import

from __future__ import absolute_import, division, print_function, unicode_literals
import warnings

warnings.filterwarnings('ignore')
import pandas as pd
import requests
import numpy as np
import tensorflow_hub as hub
import os
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Dropout
# from tensorflow.keras.layers.merge import add
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda, Input, Flatten
from tensorflow.keras import Model
from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.python.framework import ops
ops.reset_default_graph()
import tensorflow.compat.v1 as tf
#To make tf 2.0 compatible with tf1.0 code, we disable the tf2.0 functionalities
tf.disable_eager_execution()
from tensorflow.keras import backend
backend.clear_session()  # For easy reset of notebook state.

from topic_clustering_models import Modeling
from sub_intents_detection import DataPreprocessing, SubIntentsModeling

# Chatting bot based on Deep Learning

<b style = "color:red">Important!</b>

**This demo could only be run with Internet Connetion. Because our dataset and part of the model is online**

Dataset url https://github.com/google-research-datasets/Taskmaster/raw/master/TM-1-2019

In [7]:
__author__ = ["Haolin Pan", "Riade Benbaki"]
__version__ = "École Polytechnique, 2020/2/21"

In [8]:
__data__ = "https://github.com/google-research-datasets/Taskmaster/raw/master/TM-1-2019/self-dialogs.json"

## Topic Clustering

In [4]:
s = Modeling()
s.dataPreprocessing()
s.modelBuildingAndTraining()

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In this section, we try to detect an opening phrase of one scenario, one conversation; 

Normally, this would be a phrase to express a need. For example, <i>" I want a uber to the airport "</i>

Otherwise if the phrase has no information about the scenario, for expample, <i>\" Ok, that's it \"</i> we regard this as  non-opening

Then between two opening phrases, we have a more stable scenario which allow us to make detection more precise.

**The precision based on a 0.2 percentage validation set is up to 93%**

In [5]:
print("Test the models of topic clustering: ")
print(64 * "=")
test_phrases = [["Where is the nearest Starbucks ?"],["i need to repair my car"],\
                ["I need a ride from home"],["I want to order something to eat"],["can you activate"],\
                ["I want a table in center city"], ["Ok that's it!"]]
res = s.prediction(test_phrases)
for i in range(len(res)):
    print("The predicted topic of \"{} \" is : {}".format(test_phrases[i][0], res[i]))
print(64 * "-")

Test the models of topic clustering: 
The predicted topic of "Where is the nearest Starbucks ? " is : coffee
The predicted topic of "i need to repair my car " is : auto
The predicted topic of "I need a ride from home " is : uber
The predicted topic of "I want to order something to eat " is : pizza
The predicted topic of "can you activate " is : non-opening
The predicted topic of "I want a table in center city " is : restaurant
The predicted topic of "Ok that's it! " is : non-opening
----------------------------------------------------------------


## Subintents Detection

In [6]:
dp = DataPreprocessing()
i2phr = dp.intents_clustering()
SIM = SubIntentsModeling(i2phr)
if not SIM.trained():
    SIM.training_models("restaurant")
    SIM.training_models("movie")
    SIM.training_models("auto")
    SIM.training_models("pizza")
    SIM.training_models("uber")
    SIM.training_models("coffee")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


After detecting a opening phrase of one topic, we try to detect the intents more specific for the topic

For example, in the topic of restaurant reservation, we would like to get the informations like <i>time </i>, <i>location</i> etc.

Luckily, the entities which reveals this information is labeled in our data set. 

For example, in the phrase, <i>Thank you, 8 pm will be fine.</i>, <i>8 pm</i> is marked out as the time information

In our model, we try to detect the if the phrase has some information we need, if yes, what type is it.

**The precision of this model is about 0.7 to 0.8**

In [7]:
print("Test the models of Sub intent detection: ")
print(64 * "=")
test_phrases = np.array([["Where is the nearest restaurant?"], ["This Evening"], ["I want this"],
                  ["We have totally 4"], ["My dad, mom, me and my sister"], ["Do you have some recommends"],
                  ["Thank you! Bye"]])
res = SIM.prediction("restaurant", test_phrases)
for i in range(len(res)):
    print("The predicted intent of \"{}\" is {}".format(test_phrases[i][0], res[i]))
print(64 * "-")

Test the models of Sub intent detection: 
The predicted intent of "Where is the nearest restaurant?" is location
The predicted intent of "This Evening" is time
The predicted intent of "I want this" is others
The predicted intent of "We have totally 4" is num
The predicted intent of "My dad, mom, me and my sister" is num
The predicted intent of "Do you have some recommends" is type
The predicted intent of "Thank you! Bye" is name
----------------------------------------------------------------


__TO DO__

## Information Obtaining

## Response Generation

## Conversation Demo

topic allowed: 'auto repair', 'coffee ordering', 'movie booking', 'pizza ordering','restaurant reservation','uber ordering'

In [43]:
class Demo_Conversation(object):
    def __init__(self, model_cl, models_si):
        self.model_cl = model_cl
        self.models_si = models_si
        self.classes_arr = ['auto', 'coffee', 'movie', 'non-opening', 'pizza','restaurant','uber']
        self.opening_response = {"auto" : "\tIt seems you want to repair your vehicle", \
                            "coffee" : "\tIt seems you want to order some coffee", \
                            "movie" : "\tIt seems you want to book movie tickets", \
                            "pizza" : "\tIt seems you want to order some pizza",\
                            "restaurant": "\tIt seems you want to book a table",\
                            "uber": "\tIt seems you want to take ride"}

    def inConversationDetected(self):
        
        print("Conversation Test")
        print("Still working on it, not completed")
        print("Input \'~\' to stop")
        print(64*"=")
        print("\tHello, what can I do for you?")
        print(64 * "-")
        userInput = ""
        current_topic = "non-opening";
        userInput = input()
        
        while current_topic == "non-opening":
            prediction = self.model_cl.predict(np.array([[userInput], [userInput]]))
            class_pred = self.classes_arr[np.argmax(prediction[0],axis=0)]
            current_topic = class_pred
            
        print("\ttopic predicted: ", class_pred)
        print("\tentering the scenario ", class_pred)
        if class_pred not in self.classes_arr:
            print("Error, there is no class")
            return -1
        print(self.opening_response[class_pred])

        
        while (userInput != "~"):  
            subintent = self.models_si.prediction(current_topic, np.array([[userInput],[userInput]]))[0]
            print("\tIt seems that you want to say something about: ", subintent)    
            print(64* "-")
            userInput = input()
        
        print("Goodbye, thank you for your attention")

In [44]:
dc = Demo_Conversation(s.model_lstm, SIM)

In [45]:
dc.inConversationDetected()

Conversation Test
Still working on it, not completed
Input '~' to stop
	Hello, what can I do for you?
----------------------------------------------------------------
I want to see some newest movies
	topic predicted:  movie
	entering the scenario  movie
	It seems you want to book movie tickets
	It seems that you want to say something about:  name
----------------------------------------------------------------
this evening
	It seems that you want to say something about:  time
----------------------------------------------------------------
Jusr in Paris
	It seems that you want to say something about:  location
----------------------------------------------------------------
We have 4 persons totally, my parents , me and my sister
	It seems that you want to say something about:  num
----------------------------------------------------------------
Wonder woman 2 would be great
	It seems that you want to say something about:  name
---------------------------------------------------------

In [46]:
dc.inConversationDetected()

Conversation Test
Still working on it, not completed
Input '~' to stop
	Hello, what can I do for you?
----------------------------------------------------------------
I think my car is broken
	topic predicted:  auto
	entering the scenario  auto
	It seems you want to repair your vehicle
	It seems that you want to say something about:  reason
----------------------------------------------------------------
There was a little accident
	It seems that you want to say something about:  reason
----------------------------------------------------------------
It's a old car, it is 20 years old
	It seems that you want to say something about:  name
----------------------------------------------------------------
It's a BMW
	It seems that you want to say something about:  name
----------------------------------------------------------------
Could you order repair work next week
	It seems that you want to say something about:  date
----------------------------------------------------------------
Th

In [48]:
dc.inConversationDetected()

Conversation Test
Still working on it, not completed
Input '~' to stop
	Hello, what can I do for you?
----------------------------------------------------------------
I want to go to airport
	topic predicted:  uber
	entering the scenario  uber
	It seems you want to take ride
	It seems that you want to say something about:  location
----------------------------------------------------------------
As soon as possible
	It seems that you want to say something about:  time
----------------------------------------------------------------
We have 5 persons
	It seems that you want to say something about:  num
----------------------------------------------------------------
My father, my mother, me and my sister
	It seems that you want to say something about:  num
----------------------------------------------------------------
How long could it tak
	It seems that you want to say something about:  type
----------------------------------------------------------------
How long could it takes
	It 

In [50]:
dc.inConversationDetected()

Conversation Test
Still working on it, not completed
Input '~' to stop
	Hello, what can I do for you?
----------------------------------------------------------------
Book a restaurant for me
	topic predicted:  restaurant
	entering the scenario  restaurant
	It seems you want to book a table
	It seems that you want to say something about:  num
----------------------------------------------------------------
Two people, my wife and me
	It seems that you want to say something about:  num
----------------------------------------------------------------
Just around the city center
	It seems that you want to say something about:  location
----------------------------------------------------------------
I want some real good sea food
	It seems that you want to say something about:  others
----------------------------------------------------------------
this evening at 20:00
	It seems that you want to say something about:  time
----------------------------------------------------------------
~